Waterfall Retention Report and Cohort Analysis

#Import Libraries and Dataset


In [ ]:
pip install gspread google-auth pandas boto3 pyspark

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import gspread
import pyspark
from google.oauth2 import service_account
from google.cloud import bigquery
import google.auth
import pandas_gbq
import boto3

In [ ]:
spreadsheet_key = "1mDBIF7j9oH20437Eg37V58Nr2lQtSRE_lze3ULhgeZI"
scope = "https://spreadsheets.google.com/feeds"
credentials = service_account.Credentials.from_service_account_file("./projchallenge-62bce2c85e92.json")
sa = gspread.service_account(filename="./projchallenge-62bce2c85e92.json")
worksheet = sa.open("subscription_raw").sheet1
data = worksheet.get_values()
df = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
df['customer_id'] = np.arange(0, len(df))
#loop thru df, if subs is active, put presumptive end date, and tag active,
#if not active (ie. canceltime is defined, that is the end date)


False

In [ ]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50042 entries, 0 to 50041
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   subscription_started    50042 non-null  datetime64[ns]
 1   cancel_time             50042 non-null  object        
 2   subscription_plan_type  50042 non-null  object        
 3   customer_id             50042 non-null  int64         
 4   year-month              50042 non-null  period[M]     
 5   is_active               50042 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(2), period[M](1)
memory usage: 2.0+ MB


In [ ]:
#create monthly cohorts in datetime format
#df['subscription_started'] = pd.to_datetime(df['subscription_started'])
#df['year-month'] = df['subscription_started'].dt.to_period('M')
#df.head()

,subscription_started,cancel_time,subscription_plan_type,customer_id,year-month
0,2024-07-02 07:34:22,2024-09-29 18:27:48,Attachment-Bootcamp,0,2024-07
1,2024-10-21 23:44:53,,Attachment-Bootcamp,1,2024-10
2,2024-09-03 16:56:34,,Attachment-Bootcamp,2,2024-09
3,2024-08-17 10:09:49,2024-08-17 10:27:27,Attachment-Bootcamp,3,2024-08
4,2024-07-21 20:16:27,,Attachment-Bootcamp,4,2024-07


In [ ]:
#group longer term subscriptions together
# single-course, lifetime - 24 months
# monthly, attachment - 1 month
#quarterly, semi-annually, annually - 3,6,12 months respectively
df['long_subscriptions'] = df.value_counts()['single-course','lifetime']
df['monthly_subs'] = df.value_counts()['monthly', 'Attachment-Bootcamp']


<ipython-input-10-dbe4c23bbcf8>:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  df['long_subscriptions'] = df.value_counts()['single-course','lifetime']


KeyError: 'single-course'

In [ ]:
def get_date(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    return year, month

In [ ]:

sub_started_year,sub_started_month  = get_date(df, 'subscription_started')


current_year = dt.datetime.now().year
current_month = dt.datetime.now().month

diff_year = current_year - sub_started_year
diff_month = current_month - sub_started_month

df["Month_index"] = diff_year * 12 + diff_month +1
df.head()



,subscription_started,cancel_time,subscription_plan_type,year-month,Month_index
0,2024-07-02 07:34:22,2024-09-29 18:27:48,Attachment-Bootcamp,2024-07,5
1,2024-10-21 23:44:53,,Attachment-Bootcamp,2024-10,2
2,2024-09-03 16:56:34,,Attachment-Bootcamp,2024-09,3
3,2024-08-17 10:09:49,2024-08-17 10:27:27,Attachment-Bootcamp,2024-08,4
4,2024-07-21 20:16:27,,Attachment-Bootcamp,2024-07,5


In [ ]:
cohort = df.groupby(['Month_index', 'subscription_plan_type'])
cohort.head()


,subscription_started,cancel_time,subscription_plan_type,year-month,Month_index
0,2024-07-02 07:34:22,2024-09-29 18:27:48,Attachment-Bootcamp,2024-07,5
1,2024-10-21 23:44:53,,Attachment-Bootcamp,2024-10,2
2,2024-09-03 16:56:34,,Attachment-Bootcamp,2024-09,3
3,2024-08-17 10:09:49,2024-08-17 10:27:27,Attachment-Bootcamp,2024-08,4
4,2024-07-21 20:16:27,,Attachment-Bootcamp,2024-07,5
...,...,...,...,...,...
48169,2024-10-01 08:38:54,,single_course,2024-10,2
48170,2024-10-01 15:59:20,,single_course,2024-10,2
48171,2024-10-01 16:00:38,,single_course,2024-10,2
48172,2024-10-01 19:40:11,,single_course,2024-10,2


In [ ]:
#morechatgpt ass code
def is_active(sub_started, cancel_time, cohort):
  if pd.isna(cancel_time):
    return True
  diff = (cancel_time.year - sub_started.year) * 12 + (cancel_time.month - sub_started.month) +1
  return cohort <= diff
  df['active'] = df.apply(lambda x: is_active(x['subscription_started'], x['cancel_time'], x['Month_index']), axis=1)

In [ ]:
import pandas as pd
import datetime as dt

# Assuming your DataFrame is named df, and has columns 'subscription_started', 'cancel_time', and 'subscription_plan_type'
#chatgpt ass code lame ass shit doesnt even work
# Step 1: Convert subscription started to datetime format and create cohort month
df['subscription_started'] = pd.to_datetime(df['subscription_started'])
df['CohortMonth'] = df['subscription_started'].dt.to_period('M')  # Year-Month cohort identifier

# Step 2: Calculate the Month_Index for each subscription
# Calculate year and month of subscription started
df['subscription_year'] = df['subscription_started'].dt.year
df['subscription_month'] = df['subscription_started'].dt.month

# Calculate the difference in months from the cohort start
current_year = dt.datetime.now().year
current_month = dt.datetime.now().month

# Get the relative index (e.g., how many months after subscription started)
diff_year = current_year - df['subscription_year']
diff_month = current_month - df['subscription_month']

df['Month_Index'] = diff_year * 12 + diff_month + 1
#Step 2.5: calculate total subscriptions by cohort month

# Step 3: Aggregate data by CohortMonth and Month_Index
cohort_group = df.groupby(['CohortMonth', 'Month_Index']).size().reset_index(name='TotalSubscriptions')

# Step 4: Pivot the data to create the cohort matrix
cohort_matrix = cohort_group.pivot(index='CohortMonth', columns='Month_Index', values='TotalSubscriptions')

# Step 5: Calculate retention rates
cohort_sizes = cohort_matrix.iloc[:, 0]  # The size of each cohort is the first column in the matrix
retention_matrix = cohort_matrix.divide(cohort_sizes, axis=0) * 100

# Step 6: Display the retention matrix
print(retention_matrix)


Month_Index     1   2   3   4   5   6   7   8   9   10  ...  54  55  56  57  \
CohortMonth                                             ...                   
2019-09        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2019-10        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2019-11        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2019-12        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2020-01        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
...            ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..   
2024-07        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2024-08        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2024-09        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2024-10        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN NaN   
2024-11      100.0 NaN NaN NaN NaN NaN NaN NaN NaN N

In [ ]:
#transform the df, then load into bq
from datetime import datetime, timedelta
import numpy as np

def pds_transform():
  #process data, sort by date, push to BQ
  df['subscription_started'] = pd.to_datetime(df['subscription_started'], errors='coerce')
  df['cancel_time'] = pd.to_datetime(df['cancel_time'], errors='coerce')
  #sort in sequential order
  df.sort_values(by="subscription_started", inplace=True)
  #define cohorts, if ongoing, adjust cancel_time to reflect
  current_date = pd.to_datetime(datetime.now())
  df['presumptive_cancel_time'] = df['cancel_time'].fillna((current_date) + pd.offsets.MonthEnd(0))
  sub_data = []

  for index, row in df.iterrows():
    start_date = row['subscription_started']
    presumptive_cancel_time = row['presumptive_cancel_time']
    #duration of active subscription
    max_months = min(12, (presumptive_cancel_time.year - start_date.year) *12 + (presumptive_cancel_time.month - start_date.month + 1))
    for month_offset in range(max_months):
      active_month = start_date + pd.DateOffset(months=month_offset)
      sub_data.append({
        'subscription_started': start_date.strftime('%Y-%m'),
        'subscription_plan_type': row['subscription_plan_type'],
        'active_month': active_month.strftime['%Y-%m'],
        'month_number': month_offset + 1
      })

  transformed_df = pd.DataFrame(sub_data)
  retention_periods = transformed_df.groupby(['subscription_started', 'subscription_plan_type', 'month_number']).size().reset_index(name='active_subscriptions')

  retention_arr = retention_periods.pivot_table(
      index=['subscription_started', 'subscription_plan_type'],
      columns='month_number',
      values='active_subscriptions',
      fill_value=0).reset_index()

  retention_arr = retention_arr.rename(columns=lambda x: f'Month_{x}' if isinstance(x, int) else x)
  full_cohorts = retention_arr.loc[retention_arr[[f'Month{i}'for i in range(1,13)]].min(axis=1)>0]

  project_id = "projchallenge"
  table_id = "subscription_sorted.round2_technical_interview_sorted_data"
  retention_arr.to_gbq(table_id, project_id=project_id, if_exists="replace", credentials=credentials)



TypeError: 'method' object is not subscriptable